# Action Client node with Jupyter 

### Import

In [ ]:
import rospy
import assignment_2_2023.msg
from assignment_2_2023.msg import GoalCoord
from assignment_2_2023.srv import CancelGoal, CancelGoalResponse
from turtlesim.msg import Pose
from nav_msgs.msg import Odometry
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

### New Imports

In [ ]:
#import jupyros as jr
#from jupyros import ros3d
import os
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

### Init of the node

In [ ]:
time.sleep(1)

# Initialize the node
rospy.init_node('user_interface')



# Goal insert by the user publisher
pubgoal = rospy.Publisher('/goal_coord', GoalCoord, queue_size=1)

# wait for the service to be available
rospy.wait_for_service('/cancel_goal')
cancgoalServ = rospy.ServiceProxy('/cancel_goal', CancelGoal)

welcText = widgets.Textarea(value='Hi, welcome to the robot planner simulation!!', layout=widgets.Layout(width='auto'))
display(welcText)

goalSet = [] # All goal set by the user
goalCanc = [] # All goal canceled by the user
goalReached = [] # All goal reached by the robot

# response = cancel_goal_service() to call it 

## Odom subscription
Subscriber to check if the goal has been reached
change the boolean with a check between the length oh goal_reached and the number of goalSet deleting the boolean that doesn't work

In [ ]:
def goal_reach_callback (msg):
    global goalReach

    if goalSet and goalReach == False:
        if msg.pose.pose.position.x - 0.5 < goalSet[-1][0] < msg.pose.pose.position.x + 0.5 and msg.pose.pose.position.y - 0.5 < goalSet[-1][1] < msg.pose.pose.position.y + 0.5:
            goalReached.append(goalSet[-1])
            print(goalReach)
            goalReach = True
            

subpos = rospy.Subscriber('/odom', Odometry, goal_reach_callback)

### Init of the user Interface
Starting the interaction with the user throuh widgets to let him set the goal

In [ ]:
introText = widgets.Textarea(value='Please choose the goal position you want to reach with the robot.', 
                             layout=widgets.Layout(width='auto'))
display(introText)
xdes = widgets.Text(value='Choose the x coordinate:')
xgoal = widgets.IntSlider(
    value = 0,
    min = -8,
    max = 8,
    step = 1,
    layout=widgets.Layout(width='auto')
)
display(xdes, xgoal)
ydes = widgets.Text(value='Choose the y coordinate:')
ygoal = widgets.IntSlider(
    value = 0,
    min = -8,
    max = 8,
    step = 1,
    layout=widgets.Layout(width='auto')
)
display(ydes, ygoal)
bttgoal = widgets.Button(description='Send goal', 
                         layout=widgets.Layout(width='auto', display='flex', justify_content='center'), 
                         style=widgets.ButtonStyle(button_color='red', font_weight='bold', color='white'))
box = widgets.Box([bttgoal])
box.layout = widgets.Layout(display='flex', justify_content='center', width='100%')
out = widgets.Output()
display(box, out)

# Function to send the goal to the robot (modifica i print)
def on_button_clicked_set(b):
    global goalReach
    with out:
        if xgoal.value == None or ygoal.value == None:
            print('Please insert a valid number')
        else:
            goal = GoalCoord()
            goal.coord_x = xgoal.value
            goal.coord_y = ygoal.value
            pubgoal.publish(goal)
            goalSet.append([xgoal.value, ygoal.value])
            goalReach = False
            print('Goal sent to the robot')

bttgoal.on_click(on_button_clicked_set)


Interaction with the user throuh widgets to let him cancel the goal

Check the service and the serviceproxy and understand how to do it to have the return string to know the status of the request and send it back

In [ ]:
cancText = widgets.Textarea(value='Here you can cancel the the latest goal', 
                             layout=widgets.Layout(width='auto'))
display(cancText)
bttcanc = widgets.Button(description='Cancel goal', 
                         layout=widgets.Layout(width='auto', display='flex', justify_content='center'), 
                         style=widgets.ButtonStyle(button_color='purple', font_weight='bold', color='black'))
boxcanc = widgets.Box([bttcanc])
boxcanc.layout = widgets.Layout(display='flex', justify_content='center', width='100%')
out = widgets.Output()
display(boxcanc, out)

def on_button_clicked_canc(b):
    global goalReach
    with out:
        if len(goalSet) == 0:
            print('No goal to cancel')
        else:
            status = cancgoalServ()
            # perform the control
            print("status = ", status.stat)
            if (status.stat == "Done"):
                goalCanc.append(goalSet[-1])
                goalReach = True
                print('Goal cancelled')
            elif (status.stat == "Reached"):
                print('Goal not cancelled becouse already reached')
            else:
                print('Goal not cancelled, errror occoured')


bttcanc.on_click(on_button_clicked_canc)


Ass IIA: Try using widgets to let the user know the position of the robot and all targets that have been set and
cancelled in the environment

Ass IIB: by using the FuncAnimation for plotting the robot
position
- Add also a plot for the number of reached/not-reached targets
 
In the end, the notebook should have:
- an interface to assign (or cancel) goals to the robot
- a plot with the robot’s position and targets’ positions in the environment
- a plot for the number of reached/not-reached targets

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import rospy
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

def ensure_same_length(list1, list2):
    if len(list1) > len(list2):
        list1 = list1[:len(list2)]
    elif len(list2) > len(list1):
        list2 = list2[:len(list1)]
    return list1, list2

class Visualiser:
	def __init__(self):
		self.fig, self.ax = plt.subplots()
		self.ln, = plt.plot([], [], 'ro')
		self.x_data, self.y_data = [] , []

	def plot_init(self):
		self.ax.set_xlim(-10, 10)
		self.ax.set_ylim(-10, 10)
		return self.ln,
		
	def odom_callback(self, msg):
		self.y_data.append(msg.pose.pose.position.y)
		self.x_data.append(msg.pose.pose.position.x)

	def update_plot(self, frame):
		self.x_data, self.y_data = ensure_same_length(self.x_data, self.y_data)
		self.ln.set_data(self.x_data, self.y_data)
		return self.ln,

# ATTENZIONE
Migliora il grafico con tutte cosee più belline
Mostra i valori dele coordinate

Mostra le tre liste
Mostra grafico per le tre liste


In [ ]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)